In [3]:
import pandas as pd
import numpy as np

# Part1: Building the Postal Code Data Frame

In [74]:
#Get Data and populate data frame
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
dfs1 = pd.read_html(url, attrs={'class': 'wikitable'})
postal_code_df = pd.DataFrame(dfs1[0])

In [75]:
#Ignore cells with a borough that is Not assigned
postal_code_df = postal_code_df[postal_code_df.Borough != 'Not assigned']

In [76]:
#Combined neighborhoods with same postal code into one row separated with a comma
postal_code_df['Neighborhood'] = postal_code_df['Neighborhood'].str.replace(' /', ',')

In [77]:
#If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
postal_code_df['Neighborhood'] = pd.np.where(postal_code_df.Neighborhood.str.contains("Not assigned"), postal_code_df.Borough, postal_code_df.Neighborhood)

In [79]:
postal_code_df.shape

(103, 3)